In [1]:
import numpy as np
import cv2
import os
from os import listdir

from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny
from skimage import data

from pylab import imread, imshow, gray, mean
import matplotlib.pyplot as plt
import math
import PIL
from PIL import Image
from IPython.display import Image as pyImage
from IPython.display import display
import heapdict
from io import BytesIO

In [2]:
folder_dir = "/Users/27abd/Desktop/GPIP/Segmented/CS5"
imageNum = 0

numSkips = input("How many images to skip: ") 
if numSkips.isnumeric(): 
    numSkips = int(numSkips) 

How many images to skip: 0


In [ ]:
for image_name in os.listdir(folder_dir): 
    save_dir = "/Users/27abd/Desktop/GPIP/Corrected_Images/CS5"
    imageNum = imageNum + 1 
    print ("Image #", imageNum) 
    if (imageNum < numSkips): 
        continue
    image = cv2.imread("Segmented/CS5/" + image_name)
    plt.imshow(image)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    # Use canny edge detection
    edges = cv2.Canny(gray,50,150,apertureSize=3)

    # Apply HoughLinesP method to
    # to directly obtain line end points
    lines_list =[]
    lines = cv2.HoughLinesP(
                edges, # Input edge image
                1, # Distance resolution in pixels
                np.pi/180, # Angle resolution in radians
                threshold=42, # Min number of votes for valid line
                minLineLength=35, # Min allowed length of line
                maxLineGap=15 # Max allowed gap between line for joining them
                )

    if lines is None:
        continue
    maxDistance = 0
    line_q = []

    # Iterate over points
    for points in lines:
          # Extracted points nested in the list
        x1,y1,x2,y2=points[0]
        distance = math.sqrt( ((x2-x1)**2) + ((y2 - y1)**2) )
        angle = -(math.atan( (x2 - x1) / (y2 - y1) ) * ( 180 / math.pi ))
        line_q.append((distance, angle))

        # On the original image
        cv2.line(image,(x1,y1),(x2,y2),(0,255,0),2)
        # Maintain a simples lookup list for points
        lines_list.append([(x1,y1),(x2,y2)])
    line_q.sort(reverse=True)

    save_angle = False
    Original_Image = Image.open("Segmented/CS5/" + image_name)
    if(not line_q):
        Original_Image.show()
    skip = False

    while (save_angle == False and line_q and skip == False):
        currAngle = line_q.pop(0)[1]
        rotated_image = Original_Image.rotate(currAngle)
        rotated_image.show()
        userInput = input("save angle? (y/n)(s to skip) (i to input)")

        if (userInput == "y"):
            save_angle = True
        if (userInput == "s"):
            skip = True
        if (userInput == "i"):
            break
    if (skip == True):
        continue
    if (userInput != "y" and (not line_q or userInput == "i")):
        userInput = "n"
        while (userInput != "y"):
            angle = input("what angle?: ")
            if angle.isnumeric():
                angle = int(angle)
            else:
                continue
            rotated_image = Original_Image.rotate(angle)
            rotated_image.show()
            userInput = input("good? (y/n)")
    save_name = "/Corrected" + image_name
    
        # Save the result image
    save_dir = save_dir + save_name
    rotated_image.save(save_dir)

Image # 1
